<a href="https://colab.research.google.com/github/dushyantsinghraghav/Data_engineering/blob/main/ELT_using_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   # ASSIGNMENT 2B
   # DUSHYANT SINGH RAGHAV

# Streaming application using Spark Structured Streaming

# Creating the spark session with two processors
# 2.1
Importing pyspark and creating spark session .
we create the dataschema for pedestrian count and sensor location csv files similar to asssignment 2A\
we will use only two local processors instead of using all the process

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=a81cf7f28c7bfaf3b620a4173491bd79ea12a0ebd75b561528084e5b95314483
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:

import pyspark
from pyspark import SparkConf
master="local[2]"
app_name="Assignment 2B solution"
spark_conf=SparkConf().setMaster(master).setAppName(app_name)
from pyspark.sql import SparkSession
from pyspark import SparkContext
#getting or instatiating spark context object
sc=SparkContext.getOrCreate(spark_conf)
sc.setLogLevel('ERROR')
#we create spark object
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
print("the first part is implemented")


the first part is implemented


# Creating data schema for pedestrian count dataframe
# 2.2
Defining the data schema

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp, date_format
spark.conf.set('spark.sql.session.timeZone', 'UTC')



# define  data types for schema for two files pedestrian count and sensor location
pd_count_schema = StructType([
StructField("ID", IntegerType(), True),
StructField("Date_Time", StringType(), True),
StructField("Year", IntegerType(), True),
StructField("Month", StringType(), True),
StructField("Mdate", IntegerType(), True),
StructField("Day",StringType(),True),
StructField("Time", IntegerType(), True),
StructField("Sensor_ID", IntegerType(), True),
StructField("Sensor_Name", StringType(), True),
StructField("Hourly_Counts", IntegerType(), True)
])
display(pd_count_schema)


StructType([StructField('ID', IntegerType(), True), StructField('Date_Time', StringType(), True), StructField('Year', IntegerType(), True), StructField('Month', StringType(), True), StructField('Mdate', IntegerType(), True), StructField('Day', StringType(), True), StructField('Time', IntegerType(), True), StructField('Sensor_ID', IntegerType(), True), StructField('Sensor_Name', StringType(), True), StructField('Hourly_Counts', IntegerType(), True)])

# Defining the dataschema for sensor location
In this we define the data schema for sensor location csv and then read

In [ ]:
#similarly defining the schema for the sensor location file
sensor_location_schema=StructType([
StructField("sensor_id", IntegerType(), True),
StructField("sensor_name", StringType(), True),
StructField("sensor_description", StringType(), True),
StructField("installation_date", StringType(), True),
StructField("status", StringType(), True),
StructField("note", StringType(), True),
StructField("direction_1", StringType(), True),
StructField("direction_2", StringType(), True),
StructField("latitude", FloatType(), True),
StructField("longitude", FloatType(), True),
StructField("location", StringType(), True)
])



display(sensor_location_schema)

StructType([StructField('sensor_id', IntegerType(), True), StructField('sensor_name', StringType(), True), StructField('sensor_description', StringType(), True), StructField('installation_date', StringType(), True), StructField('status', StringType(), True), StructField('note', StringType(), True), StructField('direction_1', StringType(), True), StructField('direction_2', StringType(), True), StructField('latitude', FloatType(), True), StructField('longitude', FloatType(), True), StructField('location', StringType(), True)])

# we load the respective data schema above to create independent dataframes

In [ ]:
#now we load the respective schema files and create the respective dataframes
from pyspark.sql import functions as F
pd_count= spark.read.schema(pd_count_schema).options(header=True,inferSchema=True,multiline=True,delimiter=",",mode="PERMISSIVE").csv("/content/Streaming_Pedestrian_December_counts_per_hour.csv")
pd_count.show(10)
sensor_location_count=spark.read.schema(sensor_location_schema).options(header=True,inferSchema=True,multiline=True,delimiter=",",mode="PERMISSIVE").csv("Pedestrian_Counting_System_-_Sensor_Locations.csv")
sensor_location_count.show(9)

+-------+--------------------+----+--------+-----+-------+----+---------+--------------------+-------------+
|     ID|           Date_Time|Year|   Month|Mdate|    Day|Time|Sensor_ID|         Sensor_Name|Hourly_Counts|
+-------+--------------------+----+--------+-----+-------+----+---------+--------------------+-------------+
|3435630|12/01/2020 08:00:...|2020|December|    1|Tuesday|   8|       39|        Alfred Place|           83|
|3435798|12/01/2020 11:00:...|2020|December|    1|Tuesday|  11|       12|            New Quay|           86|
|3435107|12/01/2020 12:00:...|2020|December|    1|Tuesday|   0|        4|    Town Hall (West)|           78|
|3435108|12/01/2020 12:00:...|2020|December|    1|Tuesday|   0|       17|Collins Place (So...|           10|
|3435109|12/01/2020 12:00:...|2020|December|    1|Tuesday|   0|       18|Collins Place (No...|            6|
|3435110|12/01/2020 12:00:...|2020|December|    1|Tuesday|   0|       53|  Collins St (North)|           18|
|3435111|12/01/2020

In [ ]:
pd_new=pd_count.filter((pd_count.Time<24) & (pd_count.Time>23))
pd_new.show()

+---+---------+----+-----+-----+---+----+---------+-----------+-------------+
| ID|Date_Time|Year|Month|Mdate|Day|Time|Sensor_ID|Sensor_Name|Hourly_Counts|
+---+---------+----+-----+-----+---+----+---------+-----------+-------------+
+---+---------+----+-----+-----+---+----+---------+-----------+-------------+



In [ ]:
#now we stop the above spark session
spark.stop()

# Now starting  the spark session for kafka process
# 2.3

In [ ]:
#in this  this we first read the stream of data sent by the producer and for this we create
import os
os.environ['PYSPARK_SUBMIT_ARGS']='--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,split
from pyspark.sql import functions as F
from pyspark.sql.types import *
spark1=SparkSession\
   .builder\
   .appName("pedesrtian data anlaysis")\
   .getOrCreate()

In [ ]:
#we subscribe to the pedestrian analysis using spark1
topic='pedestrian_analysis'
df=spark1\
   .readStream\
   .format("kafka")\
   .option("kafka.bootstrap.servers","127.0.0.1:9092")\
   .option("subscribe",topic)\
   .load()

In [ ]:
#now defining the schema for the readstream from the above topic
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
df=df.selectExpr( "CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
#query.stop()
df.printSchema()



root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)




# we define the schema  for the streaming data
here we define the schema for the pedesrtian count producer file and then read it


In [ ]:
#using explode to get the several columns
from pyspark.sql.types import *
pd_count_schema = ArrayType(StructType([

StructField("Day",StringType(),True),
StructField("Hourly_Counts", IntegerType(), True),
StructField("Mdate",IntegerType(),True),
StructField("Date_Time",StringType(),True),
StructField("Time",IntegerType(),True),
StructField("Sensor_Name",StringType(),True),
StructField("ID",StringType(),True),
StructField("Year",IntegerType(),True),
StructField("Sensor_ID",IntegerType(),True),
StructField("Month",StringType(),True),
StructField("ts",TimestampType(),True)
]))


In [ ]:
df = df.select(F.from_json(F.col("value").cast("string"), pd_count_schema).alias('parsed_value'))


In [ ]:
df.printSchema()


root
 |-- parsed_value: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Day: string (nullable = true)
 |    |    |-- Hourly_Counts: integer (nullable = true)
 |    |    |-- Mdate: integer (nullable = true)
 |    |    |-- Date_Time: string (nullable = true)
 |    |    |-- Time: integer (nullable = true)
 |    |    |-- Sensor_Name: string (nullable = true)
 |    |    |-- ID: string (nullable = true)
 |    |    |-- Year: integer (nullable = true)
 |    |    |-- Sensor_ID: integer (nullable = true)
 |    |    |-- Month: string (nullable = true)
 |    |    |-- ts: timestamp (nullable = true)



In [ ]:
df = df.select(F.explode(F.col("parsed_value")).alias('unnested_value'))


In [ ]:
df.printSchema()

root
 |-- unnested_value: struct (nullable = true)
 |    |-- Day: string (nullable = true)
 |    |-- Hourly_Counts: integer (nullable = true)
 |    |-- Mdate: integer (nullable = true)
 |    |-- Date_Time: string (nullable = true)
 |    |-- Time: integer (nullable = true)
 |    |-- Sensor_Name: string (nullable = true)
 |    |-- ID: string (nullable = true)
 |    |-- Year: integer (nullable = true)
 |    |-- Sensor_ID: integer (nullable = true)
 |    |-- Month: string (nullable = true)
 |    |-- ts: timestamp (nullable = true)



In [ ]:
query = df\
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

In [ ]:
query.stop()


# writing the raw data onto the parquet file and it is important for real time data
# 2.4

In [ ]:
#writing the created dataframe to the parquet file
query_file_sink = df.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "parquet/pedestrian_raw_df")\
        .option("checkpointLocation", "parquet/pedestrian_raw_df/checkpoint")\
        .start()

In [ ]:
query_file_sink.stop()


In [ ]:
#now reading the parquet file
query_file_sink_df = spark1.read.parquet("parquet/pedestrian_raw_df")
query_file_sink_df.printSchema()
query_file_sink_df.show()

root
 |-- unnested_value: struct (nullable = true)
 |    |-- Day: string (nullable = true)
 |    |-- Hourly_Counts: integer (nullable = true)
 |    |-- Mdate: integer (nullable = true)
 |    |-- Date_Time: string (nullable = true)
 |    |-- Time: integer (nullable = true)
 |    |-- Sensor_Name: string (nullable = true)
 |    |-- ID: string (nullable = true)
 |    |-- Year: integer (nullable = true)
 |    |-- Sensor_ID: integer (nullable = true)
 |    |-- Month: string (nullable = true)
 |    |-- ts: timestamp (nullable = true)

+--------------------+
|      unnested_value|
+--------------------+
|[Tuesday, 3, 1, 1...|
|[Tuesday, 27, 1, ...|
+--------------------+



In [ ]:
df.printSchema()

root
 |-- unnested_value: struct (nullable = true)
 |    |-- Day: string (nullable = true)
 |    |-- Hourly_Counts: integer (nullable = true)
 |    |-- Mdate: integer (nullable = true)
 |    |-- Date_Time: string (nullable = true)
 |    |-- Time: integer (nullable = true)
 |    |-- Sensor_Name: string (nullable = true)
 |    |-- ID: string (nullable = true)
 |    |-- Year: integer (nullable = true)
 |    |-- Sensor_ID: integer (nullable = true)
 |    |-- Month: string (nullable = true)
 |    |-- ts: timestamp (nullable = true)



# Transforming streaming data into proper format
# 2.5

In [ ]:
df_formatted = df.select(
                    F.col("unnested_value.Day").alias("Day"),
                    F.col("unnested_value.Hourly_Counts").alias("Hourly_Counts"),
                    F.col("unnested_value.Mdate").alias("Mdate"),
                    F.col("unnested_value.Date_Time").alias("Date_Time"),
                    F.col("unnested_value.Time").alias("Time"),
                    F.col("unnested_value.Sensor_Name").alias("Sensor_Name"),
                    F.col("unnested_value.ID").alias("ID"),
                    F.col("unnested_value.Year").alias("Year"),
                    F.col("unnested_value.Sensor_ID").alias("Sensor_ID"),
                    F.col("unnested_value.Month").alias("Month"),
                    F.col("unnested_value.ts").alias("ts")
                )

In [ ]:
df_formatted.printSchema()

root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: string (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [ ]:
#this query is used to check whether the values of the column are visible in the dataframe
#df_formatted is the name of the dataframe which we get after structure definition
query = df_formatted\
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()


In [ ]:
query.stop()

In [ ]:
df_formatted.printSchema()

root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: string (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [ ]:
#query to display the original dataframe values on console
query = df_formatted \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

In [ ]:
#command to stop the query
query.stop()

# creating new columns next_date,next_Mdate,next_day_week,prev_count,
# next_day_of_week
# 2.6

In [ ]:
#combining the data of each day in one batch and then writing it to the console
#converting
from pyspark.sql.functions import split,col,date_add,to_date,dayofweek,to_timestamp

df2=df_formatted.withColumn("Date_Time",to_date("Date_Time",'MM/dd/yyyy hh:mm:ss a'))





In [ ]:
#printing the dataschema
df2.printSchema()





root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: date (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [ ]:
#filtering the data
from pyspark.sql.functions import col
from pyspark.sql.types import DateType,IntegerType
from pyspark.sql import functions as F
from pyspark.sql.functions import dayofweek,to_timestamp,date_format,next_day,dayofyear,weekofyear,dayofmonth

#now we create new columns next_Mdate,prev_count,next_date,next_day_week and next_day_of_week
#using withColumn and we apply all the operations on all the df2 dataframe
#df2 is the final dataframe after all the operations

df2=df2.withColumn("prev_count",col("Hourly_Counts"))
df2=df2.withColumn("next_date",date_add(df2.Date_Time, 1))
df2=df2.withColumn("next_day_of_week",dayofweek(col('next_date')))
df2=df2.withColumn("next_day_week",weekofyear(col('next_date')))
df2=df2.withColumn("next_Mdate",dayofmonth(col("next_date")))







In [ ]:
#filtering the dataframe df4 based on time constraint of 9 until 11:55
#after applying the time fiilter on df2 dataframe we get df6 dataframe
#which is used for pipelinemodel
df6=df2.filter((col('Time')>=9) &(col('Time')<=23))

In [ ]:
#datatSchema of different dataframes
#we have two dataframes df2 and df6

df2.printSchema()

df6.printSchema()


root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: date (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- prev_count: integer (nullable = true)
 |-- next_date: date (nullable = true)
 |-- next_day_of_week: integer (nullable = true)
 |-- next_day_week: integer (nullable = true)
 |-- next_Mdate: integer (nullable = true)

root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: date (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Sensor_ID: integer (nullable =

In [ ]:
#Loadding the dataframe onto the console after creating new columns
#the dataframe we get is df2
query = df2\
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

In [ ]:
#command to stop the query
query.stop()

# Loading the pipeline and then using it for tranformation and estimation
# 2.7
we load the pipeline by imprting the PipelineModel from pyspark.ml
and then we tranform the pedestrian count dataframe

In [ ]:
#loading the pipeline model
from pyspark.ml import PipelineModel
PipelineModel=PipelineModel.load("count_estimation_pipeline_model")




In [ ]:
#now we apply the pipeline model tranformation and estimation on the train and test data
#we get the predict dataframe after transformation of df6 dataframe
#moreover pipeline is applied on df2 dataframe

predict=PipelineModel.transform(df6)

In [ ]:
#we select the respective columns such as prev_count,prediction,Time,month for pr dataframe
pr=predict.select('prev_count','prediction','next_day_week','Day')


In [ ]:
predict.printSchema()

root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: date (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- prev_count: integer (nullable = true)
 |-- next_date: date (nullable = true)
 |-- next_day_of_week: integer (nullable = true)
 |-- next_day_week: integer (nullable = true)
 |-- next_Mdate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
query = pr\
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

In [ ]:
query.stop()

# writing the pr dataframe onto the parquet which includes predictions
# after applying the pipeline
# 2.7


In [ ]:
query_file_sink_pr = predict.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "parquet/prediction_df")\
        .option("checkpointLocation", "parquet/prediction_df/checkpoint")\
        .start()

In [ ]:
query_file_sink_pr.stop()

In [ ]:
#reading the parquet file
query_file_sink_df_pr = spark1.read.parquet("parquet/prediction_df")
query_file_sink_df_pr.printSchema()
query_file_sink_df_pr.show()

root
 |-- Day: string (nullable = true)
 |-- Hourly_Counts: string (nullable = true)
 |-- Mdate: integer (nullable = true)
 |-- Date_Time: date (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Sensor_Name: string (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- next_Mdate: integer (nullable = true)
 |-- prev_count: integer (nullable = true)
 |-- next_date: date (nullable = true)
 |-- next_day_week: integer (nullable = true)
 |-- next_day_of_week: integer (nullable = true)

+--------+-------------+-----+----------+----+----+--------+-------+---------+--------------------+-------------------+----------+----------+----------+-------------+----------------+
|     Day|Hourly_Counts|Mdate| Date_Time|Time|Year|   Month|     ID|Sensor_ID|         Sensor_Name|                 ts|next_Mdate|prev_count| next_date|next_day_week|


# window creation
# 2.8.a



In [ ]:
#now using watermarking to get the Hourly_Counts
#using tolerance time and window time of 10 seconds
from pyspark.sql.functions import *
windowedCounts = predict \
    .groupBy(window(predict.Day,"5 seconds"),predict.Sensor_ID,predict.Hourly_Counts)\
    .agg(F.count(col("Time")).alias("total"))\
    .select("window","total","Sensor_ID","Hourly_Counts")


In [ ]:
#creating a query to display the data on console
query_window= windowedCounts \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("sensor_hourly_counts")\
    .trigger(processingTime='5 seconds') \
    .start()


In [ ]:
#spark query to display the count of no of hours where Hourly counts >2000 including the sensor id
spark1.sql("select * from sensor_hourly_counts ").show()

+------+-----+---------+-------------+
|window|total|Sensor_ID|Hourly_Counts|
+------+-----+---------+-------------+
+------+-----+---------+-------------+



In [ ]:
#to display the no of hours for each sensor where the Hourly_Count was greater than 2000 using the sql command

query_window.stop()

In [ ]:
#2.8.b to display the sensor longitude and latitude along with the  details of Hourly_Count>2000 on next date
#here we need to join the two dataframes on common attribute and that is sensor_id
topic='sensor_location'
df_sensor=spark1\
   .readStream\
   .format("kafka")\
   .option("kafka.bootstrap.servers","127.0.0.1:9092")\
   .option("subscribe",topic)\
   .load()
#the df sensor stream is for sensor location from which we will take location,latitude,longitude and the sensor id


In [ ]:
sensor_location_schema=StructType([
StructField("sensor_id", IntegerType(), True),
StructField("longitude", FloatType(), True),
StructField("location", StringType(), True),
StructField("latitude",FloatType(),True)
])


In [ ]:
df_sensor=df_sensor.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")


In [ ]:
df_sensor=df_sensor.select(F.from_json(F.col("value").cast("string"), sensor_location_schema).alias('parsed_value'))

In [ ]:
df_sensor = df_sensor.select(
                    F.col("parsed_value.sensor_id").alias("sensor_id"),
                    F.col("parsed_value.latitude").alias("latitude"),
                    F.col("parsed_value.longitude").alias("longitude"),
                    F.col("parsed_value.location").alias("location")

)

In [ ]:
df_sensor.printSchema()

root
 |-- sensor_id: integer (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- location: string (nullable = true)



In [ ]:

#display the data of sensor on console
query_sensor= df_sensor \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

In [ ]:
query_sensor.stop()


In [ ]:
#now we join the predict dataframe and df_sensor dataframe
joined_df = df_sensor.join( predict, df_sensor.sensor_id==predict.Sensor_ID, 'inner')\
.select(predict.Sensor_ID,'latitude','longitude','Hourly_Counts')

In [ ]:
joined_df.printSchema()


root
 |-- Sensor_ID: integer (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- Hourly_Counts: integer (nullable = true)



In [ ]:
#to display the columns of joined dataframe
query_joined = joined_df\
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()


In [ ]:
query_joined.stop()

# combining latitude and longitude with hourly _counts>2000
# 2.8.b

In [ ]:
#we filter the data frame based on Hourly count>2000 and
#we get a new data frame after applying filter
joined_df_new=joined_df.filter(col('Hourly_Counts')>2000)


In [ ]:
#Now we stream the data obtained after joining both the data frames
#we write the joined_df_new dataframe to the memory in append mode
#using the queryName of location_plot
joined_stream = joined_df_new \
    .writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("location_plot")\
    .trigger(processingTime='5 seconds') \
    .start()


In [ ]:
#to display the data in location_plot
spark1.sql("select * from location_plot ").show()


+---------+----------+---------+-------------+
|Sensor_ID|  latitude|longitude|Hourly_Counts|
+---------+----------+---------+-------------+
|        1|-37.813496|144.96515|         2172|
|        1|-37.813496|144.96515|         2343|
|        1|-37.813496|144.96515|         2213|
|       41|-37.816685| 144.9669|         2099|
|       41|-37.816685| 144.9669|         2243|
|       41|-37.816685| 144.9669|         2019|
|       41|-37.816685| 144.9669|         2120|
|       41|-37.816685| 144.9669|         2165|
|       41|-37.816685| 144.9669|         2126|
|       41|-37.816685| 144.9669|         2020|
|       41|-37.816685| 144.9669|         2004|
|       41|-37.816685| 144.9669|         2124|
|       41|-37.816685| 144.9669|         2456|
|       41|-37.816685| 144.9669|         2804|
|       41|-37.816685| 144.9669|         2831|
|       41|-37.816685| 144.9669|         2830|
|       41|-37.816685| 144.9669|         2641|
|       41|-37.816685| 144.9669|         2639|
|       41|-3

In [ ]:
#we can stop the streamining of joined_stream
joined_stream.stop()


# streaming the data from sensor_location  i.e. joined_df_new
# to the kafka stream


In [ ]:
#writing back the kafka stream

sensor_location=spark1.sql("select Sensor_ID,latitude,longitude from location_plot")

query_ds = joined_df_new \
       .writeStream \
       .format("kafka") \
       .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
       .option("topic", "location_sensor") \
       .option("checkpointLocation", "parquet/prediction_df/checkpoint")\
       .start()

In [ ]:
#to stop the stream of joined_df_new data above
query_ds.stop()